In [1]:
import os
import json
import pandas as pd
from difflib import SequenceMatcher
films = pd.read_csv("/home/umbilnm/RecSysPlayground/data/interim/5k_selected_items.csv")
not_found_path = "/home/umbilnm/RecSysPlayground/data/external/not_found"
found_path = "/home/umbilnm/RecSysPlayground/data/external/found_films"
found_films = os.listdir(found_path)
not_found_films = os.listdir(not_found_path)

### Russian Films With No Original Title Filtering

In [5]:
def save_info_from_api(movie_id: int, info: dict, folder: str = "found_films"):
    path = f"/home/umbilnm/RecSysPlayground/data/external/{folder}/{movie_id}_info.json"
    if not os.path.exists(path):
        try:
            with open(path, "w") as f:
                json.dump(info, f)
        except Exception as e:
            print(f"Ошибка при сохранении информации для movie_id: {movie_id}. Ошибка: {e}")
    else:
        print(f"Файл {path} уже существует. Сохранение отменено.")

In [49]:
for film in not_found_films:
	current_path = os.path.join(not_found_path, film)
	item_id = int(film.split('_')[0])
	with open(current_path, 'r') as f:
		data = json.load(f)

	respone_api_df = pd.DataFrame.from_dict(data['docs'])
	data = films[films['item_id'] == item_id]
	title, country, release_year = data.loc[:, ['title', 'countries','release_year']].values[0]
	try:
		item = respone_api_df[(respone_api_df['name'] == title) & 
		(respone_api_df['year'] == release_year) &
		(respone_api_df['countries'].apply(lambda x: x[0]['name']) == 'Россия')]
		info = item.iloc[0].to_dict()
		save_info_from_api(item_id, info)
	except IndexError:
		pass
	

### Check Films with the same title and year    

In [ ]:
for film in not_found_films:
	current_path = os.path.join(not_found_path, film)
	item_id = int(film.split('_')[0])
	with open(current_path, 'r') as f:
		data = json.load(f)

	respone_api_df = pd.DataFrame.from_dict(data['docs'])
	data = films[films['item_id'] == item_id]
	title, country, release_year = data.loc[:, ['title', 'countries','release_year']].values[0]
	try:
		item = respone_api_df[(respone_api_df['name'] == title) & 
		(respone_api_df['year'] == release_year) &
		(respone_api_df['countries'].apply(lambda x: x[0]['name']) == 'Россия')]
		info = item.iloc[0].to_dict()
	except IndexError:
		pass
	

In [67]:
random_file = pd.Series(not_found_films).sample(1).values[0]
random_path = os.path.join(not_found_path, random_file)
random_id = int(random_file.split('_')[0])
data = films[films['item_id'] == random_id]
with open(random_path, 'r') as f:
	api_response = pd.DataFrame.from_dict(json.load(f)['docs'])

### Check descriptions

In [6]:
paths = []
for film in not_found_films:
	current_path = os.path.join(not_found_path, film)
	cur_found_path = os.path.join(found_path, film)
	item_id = int(film.split('_')[0])
	with open(current_path, 'r') as f:
		data = json.load(f)

	respone_api_df = pd.DataFrame.from_dict(data['docs'])
	data = films[films['item_id'] == item_id]
	for desc_num, description in enumerate(respone_api_df['description']):
		matcher = SequenceMatcher(None, data['description'].values[0], description)
		similarity = matcher.ratio()

		if similarity > 0.9 and len(data['description'].values[0]) > 25 and not os.path.exists(cur_found_path):
			info = respone_api_df.loc[desc_num, :].to_dict()
			save_info_from_api(movie_id=item_id, info=info)
len(os.listdir(found_path))

3794

In [9]:
pd.Series(os.listdir(found_path)).nunique()

3794

In [12]:
not_found_films = os.listdir(not_found_path)
for film in not_found_films:
	cur_not_found_path = os.path.join(not_found_path, film)
	cur_found_path = os.path.join(found_path, film)
	if os.path.exists(cur_found_path):
		os.remove(cur_not_found_path)
len(os.listdir(found_path)), len(os.listdir(not_found_path))

(3794, 1151)